In [1]:
import tensorflow as tf
import cv2
import numpy as np
from tensorflow.keras.models import load_model

2024-12-06 17:39:44.283083: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-06 17:39:44.363008: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733486984.425111   47518 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733486984.443691   47518 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 17:39:44.528206: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import tensorflow as tf

# List physical devices
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Optionally, list details
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
class TFRecord:

    @staticmethod
    def image_to_tfrecord(image):
        # Read the image
        if not tf.is_tensor(image):
            image = tf.convert_to_tensor(image, dtype=tf.uint8)

        # Ensure the image has 3 channels (e.g., RGB)
        # Ensure the image has 3 channels (e.g., RGB)
        if len(image.shape) == 2:  # Grayscale image
            image = tf.expand_dims(image, axis=-1)  # Add channel dimension (height, width -> height, width, 1)
            image = tf.image.grayscale_to_rgb(image)
        elif image.shape[-1] != 3:
            raise ValueError("Input image must have 3 channels (RGB).")

        # Encode the image as JPEG bytes
        image_raw = tf.io.encode_jpeg(image).numpy()

        # Serialize into TFRecord format
        feature = {
            'image_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_raw])),
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        return example.SerializeToString()

    @staticmethod
    def parse_tfrecord(tfrecord):
        feature_description = {
            'image_raw': tf.io.FixedLenFeature([], tf.string),
        }
        parsed = tf.io.parse_single_example(tfrecord, feature_description)
        image = tf.io.decode_image(parsed['image_raw'], channels=3)
        image = tf.image.resize(image, (224, 224))  # Adjust size to model input
        image = image / 255.0  # Normalize if needed
        return tf.expand_dims(image, axis=0)  # Add batch dimension


In [4]:
class FundusPreprocessor:
    def __init__(self, target_diameter=900, threshold_ratio=0.05, circular_diameter=6, filter_size=3,
                 normalization_filter_size=20):
        # Initialize the two component classes
        self.roi_extractor = FundusROIextractor(target_diameter, threshold_ratio, circular_diameter)
        self.illumination_equalizer = IlluminationEqualizer(filter_size, normalization_filter_size)
        self.hist_eq = HistogramEqualizer()
        self.ad_hist = AdaptiveHistogram()

    def process(self, image):
        roi_green = self.roi_extractor.process(image)

        normalized_green = self.illumination_equalizer.normalize_image(roi_green)

        green_hist = self.hist_eq.equalize_histogram(normalized_green)

        ad_green = self.ad_hist.apply_clahe(green_hist)

        return ad_green


class FundusROIextractor:
    def __init__(self, target_diameter=900, threshold_ratio=0.05, circular_diameter=6):
        self.target_diameter = target_diameter
        self.threshold_ratio = threshold_ratio
        self.circular_diameter = circular_diameter

    # green channel extraction
    def extract_green_channel(self, image):
        return image[:, :, 1]

    def apply_global_threshold(self, image):
        max_intensity = np.max(image)
        threshold_value = int(self.threshold_ratio * max_intensity)
        _, threshold_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
        return threshold_image

    def morphological_operations(self, image):
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

        # Opening (Erosion followed by Dilation)
        opened_image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

        # Closing (Dilation followed by Erosion)
        closed_image = cv2.morphologyEx(opened_image, cv2.MORPH_CLOSE, kernel)

        return closed_image

    def resize_roi(self, image):
        # resizing using cubic interpolation
        h, w = image.shape
        current_diameter = min(h, w)
        scale_factor = self.target_diameter / current_diameter
        new_size = (int(w * scale_factor), int(h * scale_factor))
        resized_image = cv2.resize(image, new_size, interpolation=cv2.INTER_CUBIC)

        return resized_image

    def circular_corrosion(self, image):
        struct_element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (self.circular_diameter, self.circular_diameter))
        eroded_image = cv2.erode(image, struct_element)

        return eroded_image

    def apply_mask(self, image, mask):
        mask = mask.astype(np.uint8)

        # Resize mask to match the original image size if necessary
        if mask.shape != image.shape:
            mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

        # Apply bitwise_and to mask the original image
        masked_image = cv2.bitwise_and(image, image, mask=mask)
        return masked_image

    def process(self, image):
        # Step 2: Extract the green channel
        green_channel = self.extract_green_channel(image)

        # Step 3: Apply global thresholding
        thresholded_image_g = self.apply_global_threshold(green_channel)

        # Step 4: Perform morphological operations (opening and closing)
        processed_image_g = self.morphological_operations(thresholded_image_g)

        # Step 5: Resize the ROI to a uniform size (e.g., 900 pixels diameter)
        resized_roi_g = self.resize_roi(processed_image_g)

        # Step 6: Apply circular corrosion to remove edge noise
        final_roi_green = self.circular_corrosion(resized_roi_g)

        masked_green = self.apply_mask(green_channel, final_roi_green)

        return masked_green


class IlluminationEqualizer:
    def __init__(self, filter_size=3, normalization_filter_size=20):
        self.filter_size = filter_size
        self.normalization_filter_size = normalization_filter_size

    def normalize_image(self, image):
        min_val = np.min(image)
        clipped_image = image - min_val
        max_val = np.max(clipped_image)
        normalized_image = cv2.normalize(clipped_image, None, 0, 255, cv2.NORM_MINMAX)

        return normalized_image


class HistogramEqualizer:
    def __init__(self):
        pass

    def equalize_histogram(self, image):
        return cv2.equalizeHist(image)


class AdaptiveHistogram:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    def apply_clahe(self, image):
        return self.clahe.apply(image)

In [33]:
image_path = "/media/mydisk/ICDCIT/Splitdataset/2/54_left.jpeg"
image = cv2.imread(image_path)

## Inference without preprocessing

In [27]:
class_labels = {0: "No DR", 1: "Mild", 2: "Moderate", 3: "Severe", 4: "Proliferative DR"}

In [34]:
serialized_tfrecord = TFRecord.image_to_tfrecord(image)
input_image = TFRecord.parse_tfrecord(serialized_tfrecord)


In [35]:
model = load_model("/media/mydisk/ICDCIT/Diabetic Retinopathy/Diabetic-Retinopathy/best_model.keras")
prediction = model.predict(input_image)
predicted_class = np.argmax(prediction, axis=1)  # Multi-class
predicted_labels = [class_labels[i] for i in predicted_class]

print("Predictions without preprocessing:")
print("Raw probabilities:", prediction)
print("Predicted classes:", predicted_class)
print("Predicted labels:", predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predictions without preprocessing:
Raw probabilities: [[0.3894   0.005157 0.508    0.01758  0.0798  ]]
Predicted classes: [2]
Predicted labels: ['Moderate']


## Inference with preprocessing

In [36]:
preprocessor = FundusPreprocessor()
preprocessed = preprocessor.process(image)

In [37]:
serialized_tfrecord_preprocessed = TFRecord.image_to_tfrecord(preprocessed)
input_image_preprocessed = TFRecord.parse_tfrecord(serialized_tfrecord_preprocessed)

In [38]:
model_preprocessed = load_model("/media/mydisk/ICDCIT/Diabetic Retinopathy/Diabetic-Retinopathy/best_model.keras")
prediction_preprocessed = model_preprocessed.predict(input_image_preprocessed)

predicted_class_preprocessed = np.argmax(prediction_preprocessed, axis=1)  # Multi-class
predicted_labels_preprocessed = [class_labels[i] for i in predicted_class_preprocessed]

print("Predictions with preprocessing:")
print("Raw probabilities:", prediction_preprocessed)
print("Predicted classes:", predicted_class_preprocessed)
print("Predicted labels:", predicted_labels_preprocessed)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predictions with preprocessing:
Raw probabilities: [[0.0274  0.10266 0.754   0.02454 0.0911 ]]
Predicted classes: [2]
Predicted labels: ['Moderate']
